# Line RT: 1D homogeneous,
# temperature gradient

## 0) Setup

In [1]:
# Define Magritte folders

MagritteSetupFolder = '/home/frederik/Dropbox/Astro/Magritte/modules/setup/'
ProjectFolder       = '/home/frederik/MagritteProjects/Lines_1D_LTE/'

In [2]:
# Define Magritte folders

import numpy as np

from bokeh.plotting import figure, show, gridplot
from bokeh.palettes import cividis, viridis
from bokeh.io       import output_notebook
output_notebook()

from glob import glob
from sys  import path
path.insert(0, MagritteSetupFolder)

# Import from MagritteSetupFolder
from lineData import LineData, planck, relativeDifference
from setup    import setupMagritte
from spheres  import nRays
from model    import model as Model

# Set  dark bokeh theme
from bokeh.themes import built_in_themes
from bokeh.io import curdoc
curdoc().theme = 'dark_minimal'

Loading BokehJS ...

## 1) Define model

Assume a 1D model (i.e. 1 ray) with constant temperature, density and abundances and a linear velocity field, yielding a constant velocity gradient

\begin{align}
T(x)     \ &= \ 50.0 + 0.3 x, \\
\rho(x)  \ &= \ cte, \\
n_{i}(x) \ &= \ cte, \\
v(x)     \ &= \ cte,
\end{align}

where $L$ is the total length of the model and $v_{\max}$ is the maximum speed reached in the model. (Numerical values given below.) We write it in this way to ensure that velocities in the model are always much smaller than $c$, since our equations for Doppler shift are not valid otherwise.

There is CMB radiation incoming on both ends of the ray as boundary condition.

In [3]:
model = Model (dim=1)

model.ncells = 100

dens = 1.0E+12   # [m^-3]
abun = 1.0E+4    # [m^-3]
temp = 1.0E+1   # [K]
dx   = 1.0E5     # [m]
dT   = 5.0E6     # [K]
dv   = 5.0E0     # [m/s]

def TT(x):
    return temp + dT / (x+dx)

def VV(x):
    return 1.0E-5 * x

model.x = [dx*i  for i in range(model.ncells)]
model.y = [0.0   for _ in range(model.ncells)]
model.z = [0.0   for _ in range(model.ncells)]

model.density     = [dens  for _ in range(model.ncells)]
model.abundance   = [abun  for _ in range(model.ncells)]
model.temperature = [TT(x) for x in model.x]

model.vx = [VV(x) for x in model.x]
model.vy = [0.0   for _ in range(model.ncells)]
model.vz = [0.0   for _ in range(model.ncells)]

model.boundary = [0, model.ncells-1]

model.defineRays (nrays=nRays(nsides=1))

model.getNeighborLists ()

# Write new model data
model.writeInput (ProjectFolder + 'Magritte_files/')

# Run setup
setupMagritte (projectFolder = ProjectFolder, runName = '')

'/home/frederik/MagritteProjects/Lines_1D_LTE/io/18-12-11_13:11:55/'

In [4]:
# Plot model
plot_model_1 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_1.line (model.x, model.density)
plot_model_1.xaxis.axis_label = "x [m]"
plot_model_1.yaxis.axis_label = "density [m^-3]"

plot_model_2 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_2.line (model.x, model.abundance)
plot_model_2.xaxis.axis_label = "x [m]"
plot_model_2.yaxis.axis_label = "abundance [xm^-3]"

plot_model_3 = figure(plot_width=400, plot_height=250)
plot_model_3.line (model.x, model.temperature)
plot_model_3.xaxis.axis_label = "x [m]"
plot_model_3.yaxis.axis_label = "temperature [K]"

plot_model_4 = figure(plot_width=400, plot_height=250)
plot_model_4.line (model.x, model.vx)
plot_model_4.xaxis.axis_label = "x [m]"
plot_model_4.yaxis.axis_label = "velocity_x [m^-3]"

plot_model = gridplot ([[plot_model_1, plot_model_2],[plot_model_3, plot_model_4]])

show (plot_model)

### Get Magritte output

In [6]:
# Get Magritte data

ioFolders = glob(ProjectFolder + 'io/*/')
ioFolders.sort()

inputFolders  = [folder +  'input/' for folder in ioFolders]
outputFolders = [folder + 'output/' for folder in ioFolders]

lastOutput = outputFolders[-1]
lastInput  =  inputFolders[-1]

pops_files = glob(lastOutput + 'populations_0*.txt')
Jeff_files = glob(lastOutput + 'Jeff_0*.txt')
J_files    = glob(lastOutput + 'J_*.txt')
G_files    = glob(lastOutput + 'G_*.txt')
nu_files   = glob(lastOutput + 'frequencies_nu*.txt')
lnr_files  = glob(lastOutput + 'frequencies_line_nr*.txt')
eta_files  = glob(lastOutput + 'eta_0*.txt')
chi_files  = glob(lastOutput + 'chi_0*.txt')

pops_files.sort()
Jeff_files.sort()
eta_files.sort()
chi_files.sort()

pops_data = [np.loadtxt(fileName) for fileName in pops_files]
Jeff_data = [np.loadtxt(fileName) for fileName in Jeff_files]
J_data    = [np.loadtxt(fileName) for fileName in J_files]
G_data    = [np.loadtxt(fileName) for fileName in G_files]
nu_data   = [np.loadtxt(fileName) for fileName in nu_files]
lnr_data  = [np.loadtxt(fileName) for fileName in lnr_files]
eta_data  = [np.loadtxt(fileName) for fileName in eta_files]
chi_data  = [np.loadtxt(fileName) for fileName in chi_files]

# Import linedata
lineData = LineData (ProjectFolder + '/Magritte_files/linedata/hco+.dat')

### Plot output

In [7]:
# Plot functions

def color(s):
    ns = int((s_max-s_min) / s_step + 1)
    es = int((s    -s_min) / s_step)
    return viridis(ns)[es]

def legend(s):
    return f'{s}'

def bokeh_log_plot(title, x, y, xlabel, ylabel):
    return

In [8]:
s_min  = 0
s_max  = model.ncells
s_step = 10

In [9]:
# Level populations

plot = figure (title='Level populations', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = range(lineData.nlev)
    y = pops_data[0][s]
    plot.line (x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "number of the level"
plot.yaxis.axis_label = "population [m^-3]"
show (plot)

In [10]:
# Mean intensity

plot = figure (title='Total mean intensity', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = range(lineData.nrad)
    y = Jeff_data[1][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "number of the transition"
plot.yaxis.axis_label = "mean intensity J [m^-3]"
show(plot)

In [11]:
# Spectrum

plot = figure (title='Spectrum', width=700, height=500, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = nu_data[0][s]
    y =  J_data[0][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "frequencies [Hz]"
plot.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot)

In [35]:
# Flux (G) spectrum

plot = figure (title='Spectrum', width=700, height=500)
for s in range(s_min, s_max, s_step):
    x = nu_data[0][s]
    y =  G_data[0][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "frequencies [Hz]"
plot.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot)

## Analytical solution

Assuming boundary condition $B_{\nu}$ on both sides of the ray, the intensity is given by

\begin{align}
I^{+}_{\nu}(x) \ &= \ B_{\nu} e^{-\tau_{\nu'}(0,x)} + \int_{0}^{x} \text{d}x' \ \eta_{\nu'}(x') \ e^{-\tau_{\nu'}(x', x)} \\
I^{-}_{\nu}(x) \ &= \ B_{\nu} e^{-\tau_{\nu'}(x,L)} + \int_{x}^{L} \text{d}x' \ \eta_{\nu'}(x') \ e^{-\tau_{\nu'}(x, x')}
\end{align}

where the optical depth $\tau^{\pm}_{\nu}$ is defined

\begin{equation}
    \tau_{\nu}(x_{1}, x_{2}) \ = \ \int_{x_{1}}^{x_{2}} \text{d} x' \ \chi_{\nu}(x') .
\end{equation}

The mean intensity is hence given by

\begin{equation}
    J_{\nu}(x) \ = \ \frac{1}{2} \big( I^{+}_{\nu}(x) \ + \ I^{-}_{\nu}(x) \big)
\end{equation}

The frequency dependence of the emissivity and opacity only comes from the line profile

\begin{align}
    \eta_{\nu}(x) \ &= \ \eta_{ij} \phi_{\nu}, \\
    \chi_{\nu}(x) \ &= \ \chi_{ij} \phi_{\nu},
\end{align}

where we assume a Gaussian profile

\begin{equation}
	\phi_{\nu}^{ij}(x) \ = \ \frac{1}{\sqrt{\pi} \ \delta\nu_{ij}} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x)}\right)^{2}\right], \hspace{5mm} \text{where} \hspace{5mm} \delta\nu_{ij}(x) \ = \ \frac{\nu_{ij}}{c} \sqrt{ \frac{2 k_{b} T(x)}{m_{\text{spec}}} \ + \ v_{\text{turb}}^{2}(x)}.
\end{equation}

Solving the integral for the optical depth then yields

\begin{equation}
  \tau^{\pm}_{\nu}(x) \ = \ \frac{\chi}{\sqrt{\pi}} \ \int_{0}^{\ell} \text{d} l \ \frac{1}{\delta\nu_{ij}(x \pm l)} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x \pm l)}\right)^{2}\right] .
\end{equation}

In [36]:
# Set line
line = 15

In [37]:
import scipy.integrate as integrate

c     = 2.99792458E+8    # [m/s] speed of light
kb    = 1.38064852E-23   # [J/K] Boltzmann's constant
mp    = 1.6726219E-27    # [kg]  proton mass
T_CMB = 2.7254800        # [K]   CMB temperature
vturb = 0.12012E3        # [m/s] turbulent speed


def pops(T):
    return lineData.LTEpop(T) * abun

def emissivity(T):
    return lineData.lineEmissivity(pops(T))

def opacity(T):
    return lineData.lineOpacity(pops(T))

def source(T):
    return emissivity(T) / opacity(T)

def B(nu):
    return planck(T_CMB, nu)

zero = model.x[0]
L    = model.x[-1]
vmax = model.vx[-1]
nuij = lineData.frequency[line]

def dnu(x):
    return nuij/c * np.sqrt(2.0*kb*TT(x)/mp + vturb**2)

def phi(nu, x):
    return 1 / (np.sqrt(np.pi) * dnu(x)) * np.exp(-((nu-nuij)/dnu(x))**2)

def eta(nu, x):
    return emissivity(TT(x))[line] * phi(nu, x)

def chi(nu, x):
    return    opacity(TT(x))[line] * phi(nu, x)

def doppler(x1,x2):
    return 1 - (VV(x2) - VV(x1)) / c

def tau_p(nu, x, xs):
    return integrate.quad (lambda xp: chi(nu*doppler(xp,x), xp), xs, x)[0]

def tau_m(nu, x, xs):
    return integrate.quad (lambda xp: chi(nu*doppler(x,xp), xp), x, xs)[0]

def I_p(nu, x):
    result  = B(nu) * np.exp(-tau_p(nu, 0, x))
    result += integrate.quad (lambda xp: eta(nu*doppler(xp,x), xp) * np.exp(-tau_p(nu, x, xp)), 0, x)[0]
    return result

def I_m(nu, x):
    result  = B(nu) * np.exp(-tau_m(nu, x, L))
    result += integrate.quad (lambda xp: eta(nu*doppler(x,xp), xp) * np.exp(-tau_m(nu, x, xp)), x, L)[0]
    return result
    
def J(nu, x):
    return 0.5 * (I_p(nu, x) + I_m(nu, x))

def relativeError(a,b):
    return 2.0 * np.abs((a-b)/(a+b))

In [38]:
# Line

plot_model = figure(title='Line model', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 5    )
    N = int(lnr_data[0][s][line] + 5 + 1)
    # model
    x = nuij + 5 * dnu(model.x[s]) * np.linspace(-1,1,20)
    y = [J(xi, model.x[s]) for xi in x]
    plot_model.line(x, y, color=color(s))
    # data
    x = nu_data[0][s][M:N]
    y =  J_data[0][s][M:N]
    plot_model.circle(x, y, color=color(s), legend=legend(s))

plot_error = figure(title='Line error', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 5    )
    N = int(lnr_data[0][s][line] + 5 + 1)
    # error
    x = nu_data[0][s][M:N]
    y = relativeError([J(xi, model.x[s]) for xi in x], J_data[0][s][M:N])
    plot_error.circle(x, y, color=color(s), legend=legend(s))

plot = gridplot([[plot_model, plot_error]])

show(plot)

In [14]:
from os  import getcwd
from sys import path
path.insert(0, f'{MagritteSetupFolder}../linedata/bin')
path.insert(0, f'{getcwd()}/LineModel/bin')

import linedata
import LineModel

ld = linedata.LineData(lineData.nlev, lineData.nrad)
ld.irad      = lineData.irad
ld.jrad      = lineData.jrad
ld.energy    = lineData.energy
ld.weight    = lineData.weight
ld.frequency = lineData.frequency
ld.A         = lineData.A
ld.B         = lineData.B

nu   = lineData.frequency[line]
zero = model.x[0]
x    = model.x[3]
L    = model.x[-1]
dx   = (L-zero) / 2000

In [17]:
# Line

plot_model = figure(title='Line model', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 5    )
    N = int(lnr_data[0][s][line] + 5 + 1)
    # model
    ld.compute_populations_LTE (model.temperature[s])
    ld.compute_emissivity_and_opacity ()
    lm = LineModel.LineModel(ld, line, zero, L)
    x = nuij + 5 * dnu(model.x[s]) * np.linspace(-1,1,20)
    y = [lm.mean_intensity(B(nu), xi, model.x[s], dx) for xi in x]
    plot_model.line(x, y, color=color(s))
    # data
    x = nu_data[0][s][M:N]
    y =  J_data[0][s][M:N]
    plot_model.circle(x, y, color=color(s), legend=legend(s))

plot_error = figure(title='Line error', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 5    )
    N = int(lnr_data[0][s][line] + 5 + 1)
    # error
    x = nu_data[0][s][M:N]
    y = relativeError([lm.mean_intensity(B(nu), xi, model.x[s], dx) for xi in x], J_data[0][s][M:N])
    plot_error.circle(x, y, color=color(s), legend=legend(s))

plot = gridplot([[plot_model, plot_error]])

show(plot)

In [17]:


print(lm.mean_intensity (nu, zero, x, L, dx))

3.042222194706955e-37


In [14]:
nu  = nuij*1.000001

zero = model.x[0]
xx   = model.x[5]*2.4565
L    = model.x[-1]

dx = (L-zero) / 10000

print(optical_depth_p (nu, chi, nuij, zero, xx, dx))
print(tau(nu, xx-zero))

print(bcd(nu) + mean_intensity(nu, eta, chi, nuij, zero, xx, L, dx))
print(J(nu, xx))

NameError: name 'test' is not defined

ImportError: cannot import name 'LineModel'

In [5]:
ld = linedata.LineData(2,3)

In [6]:
ld.compute_emissivity_and_opacity

<bound method PyCapsule.compute_emissivity_and_opacity of <linedata.LineData object at 0x7f2e65e39880>>

In [7]:
ld.irad = [1,29,2]

In [8]:
ld.compute_populations_LTE(32.3)

In [9]:
ld.energy

array([0.00000000e+000, 2.33043967e-316])

In [10]:
ld.A

array([[2.66950546e-316, 0.00000000e+000],
       [1.94481511e-316, 0.00000000e+000]])

In [18]:
T_CMB

2.72548